# nearest neighbor preserverance
1. knn preserving evaluation

## load data

In [1]:
import torch
import math
import numpy as np

In [2]:
# hyperparameters
data_shape = (2048,)

n_hidden = 500
dim_img = 2048  # number of pixels for a gap layer
dim_z = 2

# train
n_epochs = 500
batch_size = 200
learn_rate = 0.001

### train_data

In [3]:
train_data = np.load("train_data.npy")
train_pred_labels = np.load("train_pred_labels.npy")
train_data.shape, train_pred_labels.shape

((50000, 2048), (50000,))

### test_data

In [4]:
test_data = np.load("test_data.npy")
test_pred_labels = np.load("test_pred_labels.npy")
test_data.shape, test_pred_labels.shape

((10000, 2048), (10000,))

## border preserving before DBP
1. M_<CP,CP>(original)
2. M_<CP,CP>(core)
3. M_<CP,CP>(core) rho=0
4. M_<CP,DP>(core)
5. M_<CP,CP><CP,DP>(core)
6. M_<bw_Complex>(core) k=15
7. M_<bw_Complex>(core) k=30

In [5]:
# define encoder
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# test encoder/decoder		

### M_<CP,CP>(original)

In [6]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_tptp")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_tptp


#### train

In [7]:
# number of trees in random projection forest
n_trees = 5 + int(round((train_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(train_data.shape[0]))))
# distance metric
metric = "euclidean"

In [8]:
low_train = load_encoder(train_data).cpu().numpy()

Instructions for updating:
Use tf.identity instead.


In [9]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    t = trustworthiness(train_data, low_train, n_neighbors=int(k), metric='euclidean')
    print(k,t)

10 0.9842283997639268
20 0.9842833103993436
30 0.984340082635198
50 0.9844427716732266


In [9]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=10
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:28:30 2021 Building RP forest with 16 trees
Mon Jan 25 13:28:33 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Mon Jan 25 13:28:44 2021 Building RP forest with 16 trees
Mon Jan 25 13:28:45 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.93836, 10.0, 1.0)

In [10]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:28:46 2021 Building RP forest with 16 trees
Mon Jan 25 13:28:49 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:28:51 2021 Building RP forest with 16 trees
Mon Jan 25 13:28:51 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.921, 19.0, 1.0)

In [11]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:28:53 2021 Building RP forest with 16 trees
Mon Jan 25 13:28:56 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:29:00 2021 Building RP forest with 16 trees
Mon Jan 25 13:29:00 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(6.46464, 28.0, 1.0)

In [12]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:29:03 2021 Building RP forest with 16 trees
Mon Jan 25 13:29:05 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:29:17 2021 Building RP forest with 16 trees
Mon Jan 25 13:29:17 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(12.65164, 46.0, 1.0)

#### test

In [13]:
fitting_data =  np.concatenate((train_data, test_data), axis=0)

In [14]:
# number of trees in random projection forest
n_trees = 5 + int(round((fitting_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(fitting_data.shape[0]))))
# distance metric
metric = "euclidean"

In [15]:
low_fitting = load_encoder(fitting_data).cpu().numpy()

In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    trustworthiness = trustworthiness(fitting_data, low_fitting, n_neighbors=k, metric='euclidean')
    print(k,trustworthiness)

In [16]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:29:26 2021 Building RP forest with 17 trees
Mon Jan 25 13:29:29 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Mon Jan 25 13:29:31 2021 Building RP forest with 17 trees
Mon Jan 25 13:29:31 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.9149, 9.0, 1.0)

In [17]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:29:32 2021 Building RP forest with 17 trees
Mon Jan 25 13:29:36 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:29:38 2021 Building RP forest with 17 trees
Mon Jan 25 13:29:39 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.8138, 19.0, 1.0)

In [18]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:29:40 2021 Building RP forest with 17 trees
Mon Jan 25 13:29:44 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:29:48 2021 Building RP forest with 17 trees
Mon Jan 25 13:29:49 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(6.2684, 28.0, 1.0)

In [19]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:29:51 2021 Building RP forest with 17 trees
Mon Jan 25 13:29:54 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:30:08 2021 Building RP forest with 17 trees
Mon Jan 25 13:30:08 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(12.2468, 47.0, 1.0)

### M_<CP,CP>(core)

In [10]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_core_tptp")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_core_tptp


#### train

In [11]:
# number of trees in random projection forest
n_trees = 5 + int(round((train_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(train_data.shape[0]))))
# distance metric
metric = "euclidean"

In [12]:
low_train = load_encoder(train_data).cpu().numpy()

In [13]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    t = trustworthiness(train_data, low_train, n_neighbors=int(k), metric='euclidean')
    print(k,t)

10 0.9848173464574018
20 0.9848220433064169
30 0.984871325933266
50 0.9849687419683723


In [9]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=10
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:32:07 2021 Building RP forest with 16 trees
Mon Jan 25 13:32:10 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Mon Jan 25 13:32:22 2021 Building RP forest with 16 trees
Mon Jan 25 13:32:22 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.92736, 10.0, 1.0)

In [10]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:32:23 2021 Building RP forest with 16 trees
Mon Jan 25 13:32:26 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:32:28 2021 Building RP forest with 16 trees
Mon Jan 25 13:32:29 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.90048, 19.0, 1.0)

In [11]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:32:31 2021 Building RP forest with 16 trees
Mon Jan 25 13:32:33 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:32:37 2021 Building RP forest with 16 trees
Mon Jan 25 13:32:38 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(6.41748, 28.0, 1.0)

In [12]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:32:40 2021 Building RP forest with 16 trees
Mon Jan 25 13:32:43 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:32:54 2021 Building RP forest with 16 trees
Mon Jan 25 13:32:55 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(12.58976, 46.0, 1.0)

#### test

In [13]:
fitting_data =  np.concatenate((train_data, test_data), axis=0)

In [14]:
# number of trees in random projection forest
n_trees = 5 + int(round((fitting_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(fitting_data.shape[0]))))
# distance metric
metric = "euclidean"

In [15]:
low_fitting = load_encoder(fitting_data).cpu().numpy()

In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    trustworthiness = trustworthiness(fitting_data, low_fitting, n_neighbors=k, metric='euclidean')
    print(k,trustworthiness)

In [16]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:33:03 2021 Building RP forest with 17 trees
Mon Jan 25 13:33:07 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Mon Jan 25 13:33:08 2021 Building RP forest with 17 trees
Mon Jan 25 13:33:08 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.9149, 9.0, 1.0)

In [17]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:33:10 2021 Building RP forest with 17 trees
Mon Jan 25 13:33:13 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:33:16 2021 Building RP forest with 17 trees
Mon Jan 25 13:33:16 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.797, 18.0, 1.0)

In [18]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:33:18 2021 Building RP forest with 17 trees
Mon Jan 25 13:33:22 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:33:26 2021 Building RP forest with 17 trees
Mon Jan 25 13:33:26 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(6.2076, 28.0, 1.0)

In [19]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:33:29 2021 Building RP forest with 17 trees
Mon Jan 25 13:33:32 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:33:46 2021 Building RP forest with 17 trees
Mon Jan 25 13:33:46 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(12.1519, 46.0, 1.0)

### M_<CP,CP>(core) rho=0

In [14]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_core_tptp_rho0")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_core_tptp_rho0


#### train

In [15]:
# number of trees in random projection forest
n_trees = 5 + int(round((train_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(train_data.shape[0]))))
# distance metric
metric = "euclidean"

In [16]:
low_train = load_encoder(train_data).cpu().numpy()

In [17]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    t = trustworthiness(train_data, low_train, n_neighbors=int(k), metric='euclidean')
    print(k,t)

10 0.9849535878122218
20 0.9849754818439248
30 0.9849990296503152
50 0.9850609150617432


In [23]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=10
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:35:54 2021 Building RP forest with 16 trees
Mon Jan 25 13:35:57 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Mon Jan 25 13:35:58 2021 Building RP forest with 16 trees
Mon Jan 25 13:35:58 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.96362, 9.0, 1.0)

In [24]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:36:00 2021 Building RP forest with 16 trees
Mon Jan 25 13:36:03 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:36:05 2021 Building RP forest with 16 trees
Mon Jan 25 13:36:05 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(4.00096, 19.0, 1.0)

In [25]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:36:07 2021 Building RP forest with 16 trees
Mon Jan 25 13:36:10 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:36:14 2021 Building RP forest with 16 trees
Mon Jan 25 13:36:14 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(6.59022, 29.0, 1.0)

In [26]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:36:17 2021 Building RP forest with 16 trees
Mon Jan 25 13:36:19 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:36:31 2021 Building RP forest with 16 trees
Mon Jan 25 13:36:31 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(12.87984, 46.0, 1.0)

#### test

In [27]:
fitting_data =  np.concatenate((train_data, test_data), axis=0)

In [28]:
# number of trees in random projection forest
n_trees = 5 + int(round((fitting_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(fitting_data.shape[0]))))
# distance metric
metric = "euclidean"

In [29]:
low_fitting = load_encoder(fitting_data).cpu().numpy()

In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    trustworthiness = trustworthiness(fitting_data, low_fitting, n_neighbors=k, metric='euclidean')
    print(k,trustworthiness)

In [30]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:36:40 2021 Building RP forest with 17 trees
Mon Jan 25 13:36:44 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Mon Jan 25 13:36:45 2021 Building RP forest with 17 trees
Mon Jan 25 13:36:46 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.9425, 9.0, 1.0)

In [31]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:36:47 2021 Building RP forest with 17 trees
Mon Jan 25 13:36:50 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:36:53 2021 Building RP forest with 17 trees
Mon Jan 25 13:36:53 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.8773, 19.0, 1.0)

In [32]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:36:55 2021 Building RP forest with 17 trees
Mon Jan 25 13:36:58 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:37:03 2021 Building RP forest with 17 trees
Mon Jan 25 13:37:03 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(6.333, 27.0, 1.0)

In [33]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:37:06 2021 Building RP forest with 17 trees
Mon Jan 25 13:37:09 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:37:23 2021 Building RP forest with 17 trees
Mon Jan 25 13:37:23 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(12.3911, 48.0, 1.0)

### M_<CP,DP>(core)

In [6]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_core_tpdp")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_core_tpdp


#### train

In [7]:
# number of trees in random projection forest
n_trees = 5 + int(round((train_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(train_data.shape[0]))))
# distance metric
metric = "euclidean"

In [8]:
low_train = load_encoder(train_data).cpu().numpy()

Instructions for updating:
Use tf.identity instead.


In [21]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    t = trustworthiness(train_data, low_train, n_neighbors=int(k), metric='euclidean')
    print(k,t)

10 0.9765683108963779
20 0.9765955240896947
30 0.9766400976888969
50 1.0111850059189376


In [9]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=10
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:18:06 2021 Building RP forest with 16 trees
Mon Jan 25 17:18:09 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Mon Jan 25 17:18:23 2021 Building RP forest with 16 trees
Mon Jan 25 17:18:23 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.68632, 9.0, 1.0)

In [10]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:18:25 2021 Building RP forest with 16 trees
Mon Jan 25 17:18:28 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:18:30 2021 Building RP forest with 16 trees
Mon Jan 25 17:18:30 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.08414, 19.0, 1.0)

In [11]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:18:33 2021 Building RP forest with 16 trees
Mon Jan 25 17:18:35 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:18:40 2021 Building RP forest with 16 trees
Mon Jan 25 17:18:40 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(4.87756, 28.0, 1.0)

In [12]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:18:43 2021 Building RP forest with 16 trees
Mon Jan 25 17:18:46 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:18:58 2021 Building RP forest with 16 trees
Mon Jan 25 17:18:58 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(9.27474, 45.0, 1.0)

#### test

In [13]:
fitting_data =  np.concatenate((train_data, test_data), axis=0)

In [14]:
# number of trees in random projection forest
n_trees = 5 + int(round((fitting_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(fitting_data.shape[0]))))
# distance metric
metric = "euclidean"

In [15]:
low_fitting = load_encoder(fitting_data).cpu().numpy()

In [25]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    t = trustworthiness(fitting_data, low_fitting, n_neighbors=k, metric='euclidean')
    print(k,t)

10 0.9758711358211982


TypeError: 'numpy.float64' object is not callable

In [16]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:21:27 2021 Building RP forest with 17 trees
Mon Jan 25 17:21:31 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Mon Jan 25 17:21:32 2021 Building RP forest with 17 trees
Mon Jan 25 17:21:32 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.754, 9.0, 1.0)

In [17]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:21:34 2021 Building RP forest with 17 trees
Mon Jan 25 17:21:37 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:21:40 2021 Building RP forest with 17 trees
Mon Jan 25 17:21:40 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.1878, 18.0, 1.0)

In [18]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:21:42 2021 Building RP forest with 17 trees
Mon Jan 25 17:21:45 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:21:50 2021 Building RP forest with 17 trees
Mon Jan 25 17:21:50 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(4.9968, 27.0, 1.0)

In [19]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:21:52 2021 Building RP forest with 17 trees
Mon Jan 25 17:21:56 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:22:10 2021 Building RP forest with 17 trees
Mon Jan 25 17:22:10 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(9.3461, 45.0, 1.0)

### M_<CP,CP><CP,DP>(core)

In [20]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_core_tptptpdp")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_core_tptptpdp


#### train

In [21]:
# number of trees in random projection forest
n_trees = 5 + int(round((train_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(train_data.shape[0]))))
# distance metric
metric = "euclidean"

In [22]:
low_train = load_encoder(train_data).cpu().numpy()

In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    t = trustworthiness(train_data, low_train, n_neighbors=int(k), metric='euclidean')
    print(k,t)

In [23]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=10
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:22:33 2021 Building RP forest with 16 trees
Mon Jan 25 17:22:36 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Mon Jan 25 17:22:37 2021 Building RP forest with 16 trees
Mon Jan 25 17:22:37 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.92228, 10.0, 1.0)

In [24]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:22:39 2021 Building RP forest with 16 trees
Mon Jan 25 17:22:42 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:22:44 2021 Building RP forest with 16 trees
Mon Jan 25 17:22:44 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.8684, 19.0, 1.0)

In [25]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:22:46 2021 Building RP forest with 16 trees
Mon Jan 25 17:22:49 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:22:53 2021 Building RP forest with 16 trees
Mon Jan 25 17:22:53 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(6.3445, 28.0, 1.0)

In [26]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:22:56 2021 Building RP forest with 16 trees
Mon Jan 25 17:22:59 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:23:11 2021 Building RP forest with 16 trees
Mon Jan 25 17:23:11 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(12.40648, 46.0, 1.0)

#### test

In [27]:
fitting_data =  np.concatenate((train_data, test_data), axis=0)

In [28]:
# number of trees in random projection forest
n_trees = 5 + int(round((fitting_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(fitting_data.shape[0]))))
# distance metric
metric = "euclidean"

In [29]:
low_fitting = load_encoder(fitting_data).cpu().numpy()

In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    trustworthiness = trustworthiness(fitting_data, low_fitting, n_neighbors=k, metric='euclidean')
    print(k,trustworthiness)

In [30]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:23:19 2021 Building RP forest with 17 trees
Mon Jan 25 17:23:23 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Mon Jan 25 17:23:25 2021 Building RP forest with 17 trees
Mon Jan 25 17:23:25 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.9095, 9.0, 1.0)

In [31]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:23:26 2021 Building RP forest with 17 trees
Mon Jan 25 17:23:30 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:23:32 2021 Building RP forest with 17 trees
Mon Jan 25 17:23:33 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.7786, 18.0, 1.0)

In [32]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:23:34 2021 Building RP forest with 17 trees
Mon Jan 25 17:23:38 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:23:43 2021 Building RP forest with 17 trees
Mon Jan 25 17:23:43 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(6.1705, 28.0, 1.0)

In [33]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:23:45 2021 Building RP forest with 17 trees
Mon Jan 25 17:23:49 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:24:03 2021 Building RP forest with 17 trees
Mon Jan 25 17:24:03 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(11.9782, 47.0, 1.0)

### M_<new_Complex, k=15> (core)

In [34]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_core_complex_15")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_core_complex_15


#### train

In [35]:
# number of trees in random projection forest
n_trees = 5 + int(round((train_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(train_data.shape[0]))))
# distance metric
metric = "euclidean"

In [36]:
low_train = load_encoder(train_data).cpu().numpy()

In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    t = trustworthiness(train_data, low_train, n_neighbors=int(k), metric='euclidean')
    print(k,t)

In [37]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=10
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:24:30 2021 Building RP forest with 16 trees
Mon Jan 25 17:24:32 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Mon Jan 25 17:24:34 2021 Building RP forest with 16 trees
Mon Jan 25 17:24:34 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.58952, 9.0, 1.0)

In [38]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:24:36 2021 Building RP forest with 16 trees
Mon Jan 25 17:24:38 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:24:40 2021 Building RP forest with 16 trees
Mon Jan 25 17:24:41 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(2.80488, 17.0, 1.0)

In [39]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:24:43 2021 Building RP forest with 16 trees
Mon Jan 25 17:24:45 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:24:49 2021 Building RP forest with 16 trees
Mon Jan 25 17:24:50 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(4.36568, 27.0, 1.0)

In [40]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:24:52 2021 Building RP forest with 16 trees
Mon Jan 25 17:24:55 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:25:07 2021 Building RP forest with 16 trees
Mon Jan 25 17:25:07 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(8.20522, 45.0, 1.0)

#### test

In [41]:
fitting_data =  np.concatenate((train_data, test_data), axis=0)

In [42]:
# number of trees in random projection forest
n_trees = 5 + int(round((fitting_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(fitting_data.shape[0]))))
# distance metric
metric = "euclidean"

In [43]:
low_fitting = load_encoder(fitting_data).cpu().numpy()

In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    trustworthiness = trustworthiness(fitting_data, low_fitting, n_neighbors=k, metric='euclidean')
    print(k,trustworthiness)

In [44]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:25:16 2021 Building RP forest with 17 trees
Mon Jan 25 17:25:20 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Mon Jan 25 17:25:21 2021 Building RP forest with 17 trees
Mon Jan 25 17:25:22 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.6802, 9.0, 1.0)

In [45]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:25:23 2021 Building RP forest with 17 trees
Mon Jan 25 17:25:27 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:25:29 2021 Building RP forest with 17 trees
Mon Jan 25 17:25:30 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(2.9823, 19.0, 1.0)

In [46]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:25:31 2021 Building RP forest with 17 trees
Mon Jan 25 17:25:35 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:25:40 2021 Building RP forest with 17 trees
Mon Jan 25 17:25:40 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(4.604, 24.0, 1.0)

In [47]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:25:42 2021 Building RP forest with 17 trees
Mon Jan 25 17:25:46 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:26:00 2021 Building RP forest with 17 trees
Mon Jan 25 17:26:00 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(8.4895, 44.0, 1.0)

### M_<new_Complex, k=30> (core)

In [48]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_core_complex_30")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_core_complex_30


#### train

In [49]:
# number of trees in random projection forest
n_trees = 5 + int(round((train_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(train_data.shape[0]))))
# distance metric
metric = "euclidean"

In [50]:
low_train = load_encoder(train_data).cpu().numpy()

In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    t = trustworthiness(train_data, low_train, n_neighbors=int(k), metric='euclidean')
    print(k,t)

In [51]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=10
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:26:22 2021 Building RP forest with 16 trees
Mon Jan 25 17:26:25 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Mon Jan 25 17:26:26 2021 Building RP forest with 16 trees
Mon Jan 25 17:26:26 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.48556, 10.0, 1.0)

In [52]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:26:28 2021 Building RP forest with 16 trees
Mon Jan 25 17:26:31 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:26:33 2021 Building RP forest with 16 trees
Mon Jan 25 17:26:33 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(2.50754, 19.0, 1.0)

In [53]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:26:35 2021 Building RP forest with 16 trees
Mon Jan 25 17:26:38 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:26:41 2021 Building RP forest with 16 trees
Mon Jan 25 17:26:42 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.82668, 29.0, 1.0)

In [54]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:26:44 2021 Building RP forest with 16 trees
Mon Jan 25 17:26:47 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:26:59 2021 Building RP forest with 16 trees
Mon Jan 25 17:26:59 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(7.07654, 49.0, 1.0)

#### test

In [55]:
fitting_data =  np.concatenate((train_data, test_data), axis=0)

In [56]:
# number of trees in random projection forest
n_trees = 5 + int(round((fitting_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(fitting_data.shape[0]))))
# distance metric
metric = "euclidean"

In [57]:
low_fitting = load_encoder(fitting_data).cpu().numpy()

In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    trustworthiness = trustworthiness(fitting_data, low_fitting, n_neighbors=k, metric='euclidean')
    print(k,trustworthiness)

In [58]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:27:08 2021 Building RP forest with 17 trees
Mon Jan 25 17:27:11 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Mon Jan 25 17:27:13 2021 Building RP forest with 17 trees
Mon Jan 25 17:27:13 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.6062, 9.0, 1.0)

In [59]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:27:14 2021 Building RP forest with 17 trees
Mon Jan 25 17:27:18 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:27:21 2021 Building RP forest with 17 trees
Mon Jan 25 17:27:21 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(2.7418, 18.0, 1.0)

In [60]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:27:23 2021 Building RP forest with 17 trees
Mon Jan 25 17:27:26 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:27:31 2021 Building RP forest with 17 trees
Mon Jan 25 17:27:31 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(4.1855, 27.0, 1.0)

In [61]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:27:33 2021 Building RP forest with 17 trees
Mon Jan 25 17:27:37 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:27:50 2021 Building RP forest with 17 trees
Mon Jan 25 17:27:51 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(7.6436, 44.0, 1.0)

### M_<new_Complex_symmetric, k=15> (core)

In [7]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_core_complex_symmetric_15")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_core_complex_symmetric_15


#### train

In [8]:
# number of trees in random projection forest
n_trees = 5 + int(round((train_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(train_data.shape[0]))))
# distance metric
metric = "euclidean"

In [9]:
low_train = load_encoder(train_data).cpu().numpy()

Instructions for updating:
Use tf.identity instead.


In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    t = trustworthiness(train_data, low_train, n_neighbors=int(k), metric='euclidean')
    print(k,t)

In [10]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=10
border_pres.mean(), border_pres.max(),border_pres.min()

Wed Jan 27 14:39:29 2021 Building RP forest with 16 trees
Wed Jan 27 14:39:32 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Wed Jan 27 14:39:44 2021 Building RP forest with 16 trees
Wed Jan 27 14:39:44 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.4783, 9.0, 1.0)

In [11]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Wed Jan 27 14:39:46 2021 Building RP forest with 16 trees
Wed Jan 27 14:39:48 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Wed Jan 27 14:39:50 2021 Building RP forest with 16 trees
Wed Jan 27 14:39:51 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(2.5291, 17.0, 1.0)

In [12]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Wed Jan 27 14:39:53 2021 Building RP forest with 16 trees
Wed Jan 27 14:39:55 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Wed Jan 27 14:39:59 2021 Building RP forest with 16 trees
Wed Jan 27 14:39:59 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.92082, 27.0, 1.0)

In [13]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Wed Jan 27 14:40:02 2021 Building RP forest with 16 trees
Wed Jan 27 14:40:05 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Wed Jan 27 14:40:16 2021 Building RP forest with 16 trees
Wed Jan 27 14:40:16 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(7.44558, 45.0, 1.0)

#### test

In [14]:
fitting_data =  np.concatenate((train_data, test_data), axis=0)

In [15]:
# number of trees in random projection forest
n_trees = 5 + int(round((fitting_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(fitting_data.shape[0]))))
# distance metric
metric = "euclidean"

In [16]:
low_fitting = load_encoder(fitting_data).cpu().numpy()

In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    trustworthiness = trustworthiness(fitting_data, low_fitting, n_neighbors=k, metric='euclidean')
    print(k,trustworthiness)

In [17]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Wed Jan 27 14:40:25 2021 Building RP forest with 17 trees
Wed Jan 27 14:40:29 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Wed Jan 27 14:40:30 2021 Building RP forest with 17 trees
Wed Jan 27 14:40:30 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.4806, 9.0, 1.0)

In [18]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Wed Jan 27 14:40:31 2021 Building RP forest with 17 trees
Wed Jan 27 14:40:35 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Wed Jan 27 14:40:38 2021 Building RP forest with 17 trees
Wed Jan 27 14:40:38 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(2.4941, 18.0, 1.0)

In [19]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Wed Jan 27 14:40:39 2021 Building RP forest with 17 trees
Wed Jan 27 14:40:43 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Wed Jan 27 14:40:47 2021 Building RP forest with 17 trees
Wed Jan 27 14:40:48 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.8102, 25.0, 1.0)

In [20]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Wed Jan 27 14:40:50 2021 Building RP forest with 17 trees
Wed Jan 27 14:40:53 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Wed Jan 27 14:41:07 2021 Building RP forest with 17 trees
Wed Jan 27 14:41:07 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(7.1233, 44.0, 1.0)

### M_<mixed_Complex_symmetric, k=15> (core)

In [6]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_mixed_complex_15")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_mixed_complex_15


#### train

In [7]:
# number of trees in random projection forest
n_trees = 5 + int(round((train_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(train_data.shape[0]))))
# distance metric
metric = "euclidean"

In [8]:
low_train = load_encoder(train_data).cpu().numpy()

Instructions for updating:
Use tf.identity instead.


In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    t = trustworthiness(train_data, low_train, n_neighbors=int(k), metric='euclidean')
    print(k,t)

In [9]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=10
border_pres.mean(), border_pres.max(),border_pres.min()

Wed Jan 27 16:32:17 2021 Building RP forest with 16 trees
Wed Jan 27 16:32:20 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Wed Jan 27 16:32:32 2021 Building RP forest with 16 trees
Wed Jan 27 16:32:32 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.784, 10.0, 1.0)

In [10]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Wed Jan 27 16:32:34 2021 Building RP forest with 16 trees
Wed Jan 27 16:32:36 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Wed Jan 27 16:32:39 2021 Building RP forest with 16 trees
Wed Jan 27 16:32:39 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.47878, 19.0, 1.0)

In [11]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Wed Jan 27 16:32:41 2021 Building RP forest with 16 trees
Wed Jan 27 16:32:44 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Wed Jan 27 16:32:48 2021 Building RP forest with 16 trees
Wed Jan 27 16:32:48 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(5.6864, 28.0, 1.0)

In [12]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Wed Jan 27 16:32:51 2021 Building RP forest with 16 trees
Wed Jan 27 16:32:54 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Wed Jan 27 16:33:06 2021 Building RP forest with 16 trees
Wed Jan 27 16:33:06 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(11.17822, 45.0, 1.0)

#### test

In [13]:
fitting_data =  np.concatenate((train_data, test_data), axis=0)

In [14]:
# number of trees in random projection forest
n_trees = 5 + int(round((fitting_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(fitting_data.shape[0]))))
# distance metric
metric = "euclidean"

In [15]:
low_fitting = load_encoder(fitting_data).cpu().numpy()

In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    trustworthiness = trustworthiness(fitting_data, low_fitting, n_neighbors=k, metric='euclidean')
    print(k,trustworthiness)

In [16]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Wed Jan 27 16:33:14 2021 Building RP forest with 17 trees
Wed Jan 27 16:33:18 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Wed Jan 27 16:33:20 2021 Building RP forest with 17 trees
Wed Jan 27 16:33:20 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.7262, 9.0, 1.0)

In [17]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Wed Jan 27 16:33:21 2021 Building RP forest with 17 trees
Wed Jan 27 16:33:25 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Wed Jan 27 16:33:28 2021 Building RP forest with 17 trees
Wed Jan 27 16:33:28 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.2429, 18.0, 1.0)

In [18]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Wed Jan 27 16:33:30 2021 Building RP forest with 17 trees
Wed Jan 27 16:33:34 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Wed Jan 27 16:33:38 2021 Building RP forest with 17 trees
Wed Jan 27 16:33:39 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(5.2241, 26.0, 1.0)

In [19]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Wed Jan 27 16:33:41 2021 Building RP forest with 17 trees
Wed Jan 27 16:33:45 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Wed Jan 27 16:33:59 2021 Building RP forest with 17 trees
Wed Jan 27 16:33:59 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(10.1944, 44.0, 1.0)

### M_<mixed_Complex_symmetric, k=30> (core)

In [6]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_mixed_complex_30")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_mixed_complex_30


#### train

In [7]:
# number of trees in random projection forest
n_trees = 5 + int(round((train_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(train_data.shape[0]))))
# distance metric
metric = "euclidean"

In [8]:
low_train = load_encoder(train_data).cpu().numpy()

Instructions for updating:
Use tf.identity instead.


In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    t = trustworthiness(train_data, low_train, n_neighbors=int(k), metric='euclidean')
    print(k,t)

In [9]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=10
border_pres.mean(), border_pres.max(),border_pres.min()

Wed Jan 27 19:15:48 2021 Building RP forest with 16 trees
Wed Jan 27 19:15:52 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Wed Jan 27 19:16:04 2021 Building RP forest with 16 trees
Wed Jan 27 19:16:05 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.81634, 9.0, 1.0)

In [10]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Wed Jan 27 19:16:06 2021 Building RP forest with 16 trees
Wed Jan 27 19:16:09 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Wed Jan 27 19:16:12 2021 Building RP forest with 16 trees
Wed Jan 27 19:16:12 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.61412, 18.0, 1.0)

In [11]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Wed Jan 27 19:16:15 2021 Building RP forest with 16 trees
Wed Jan 27 19:16:17 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Wed Jan 27 19:16:22 2021 Building RP forest with 16 trees
Wed Jan 27 19:16:22 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(5.97022, 28.0, 1.0)

In [12]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Wed Jan 27 19:16:25 2021 Building RP forest with 16 trees
Wed Jan 27 19:16:28 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Wed Jan 27 19:16:41 2021 Building RP forest with 16 trees
Wed Jan 27 19:16:41 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(11.77754, 46.0, 1.0)

#### test

In [13]:
fitting_data =  np.concatenate((train_data, test_data), axis=0)

In [14]:
# number of trees in random projection forest
n_trees = 5 + int(round((fitting_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(fitting_data.shape[0]))))
# distance metric
metric = "euclidean"

In [15]:
low_fitting = load_encoder(fitting_data).cpu().numpy()

In [16]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    trustworthiness = trustworthiness(fitting_data, low_fitting, n_neighbors=k, metric='euclidean')
    print(k,trustworthiness)

KeyboardInterrupt: 

In [17]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Wed Jan 27 19:18:50 2021 Building RP forest with 17 trees
Wed Jan 27 19:18:54 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Wed Jan 27 19:18:56 2021 Building RP forest with 17 trees
Wed Jan 27 19:18:56 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.7735, 9.0, 1.0)

In [18]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Wed Jan 27 19:18:58 2021 Building RP forest with 17 trees
Wed Jan 27 19:19:02 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Wed Jan 27 19:19:05 2021 Building RP forest with 17 trees
Wed Jan 27 19:19:05 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.4119, 17.0, 1.0)

In [19]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Wed Jan 27 19:19:07 2021 Building RP forest with 17 trees
Wed Jan 27 19:19:11 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Wed Jan 27 19:19:16 2021 Building RP forest with 17 trees
Wed Jan 27 19:19:16 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(5.5643, 27.0, 1.0)

In [21]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Wed Jan 27 19:19:48 2021 Building RP forest with 17 trees
Wed Jan 27 19:19:51 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Wed Jan 27 19:20:07 2021 Building RP forest with 17 trees
Wed Jan 27 19:20:07 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(10.9179, 47.0, 1.0)

### M_<mixed_symmetric_complex, k=15> (core)

In [6]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_mixed_complex_symmetric_15")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_mixed_complex_symmetric_15


#### train

In [7]:
# number of trees in random projection forest
n_trees = 5 + int(round((train_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(train_data.shape[0]))))
# distance metric
metric = "euclidean"

In [8]:
low_train = load_encoder(train_data).cpu().numpy()

Instructions for updating:
Use tf.identity instead.


In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    t = trustworthiness(train_data, low_train, n_neighbors=int(k), metric='euclidean')
    print(k,t)

In [9]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=10
border_pres.mean(), border_pres.max(),border_pres.min()

Thu Jan 28 22:05:16 2021 Building RP forest with 16 trees
Thu Jan 28 22:05:19 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Thu Jan 28 22:05:30 2021 Building RP forest with 16 trees
Thu Jan 28 22:05:30 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.79224, 10.0, 1.0)

In [10]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Thu Jan 28 22:05:32 2021 Building RP forest with 16 trees
Thu Jan 28 22:05:35 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Thu Jan 28 22:05:37 2021 Building RP forest with 16 trees
Thu Jan 28 22:05:37 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.50454, 18.0, 1.0)

In [11]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Thu Jan 28 22:05:39 2021 Building RP forest with 16 trees
Thu Jan 28 22:05:42 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Thu Jan 28 22:05:46 2021 Building RP forest with 16 trees
Thu Jan 28 22:05:46 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(5.73086, 28.0, 1.0)

In [12]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Thu Jan 28 22:05:48 2021 Building RP forest with 16 trees
Thu Jan 28 22:05:51 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Thu Jan 28 22:06:02 2021 Building RP forest with 16 trees
Thu Jan 28 22:06:03 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(11.2848, 45.0, 1.0)

#### test

In [13]:
fitting_data =  np.concatenate((train_data, test_data), axis=0)

In [14]:
# number of trees in random projection forest
n_trees = 5 + int(round((fitting_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(fitting_data.shape[0]))))
# distance metric
metric = "euclidean"

In [15]:
low_fitting = load_encoder(fitting_data).cpu().numpy()

In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    trustworthiness = trustworthiness(fitting_data, low_fitting, n_neighbors=k, metric='euclidean')
    print(k,trustworthiness)

In [16]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Thu Jan 28 22:06:11 2021 Building RP forest with 17 trees
Thu Jan 28 22:06:15 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Thu Jan 28 22:06:16 2021 Building RP forest with 17 trees
Thu Jan 28 22:06:16 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.7477, 9.0, 1.0)

In [17]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Thu Jan 28 22:06:18 2021 Building RP forest with 17 trees
Thu Jan 28 22:06:21 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Thu Jan 28 22:06:24 2021 Building RP forest with 17 trees
Thu Jan 28 22:06:24 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.354, 18.0, 1.0)

In [18]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Thu Jan 28 22:06:25 2021 Building RP forest with 17 trees
Thu Jan 28 22:06:29 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Thu Jan 28 22:06:34 2021 Building RP forest with 17 trees
Thu Jan 28 22:06:34 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(5.4236, 26.0, 1.0)

In [19]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Thu Jan 28 22:06:36 2021 Building RP forest with 17 trees
Thu Jan 28 22:06:40 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Thu Jan 28 22:06:54 2021 Building RP forest with 17 trees
Thu Jan 28 22:06:54 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(10.5396, 43.0, 1.0)

### M_<mixed_symmetric_complex, k=15> (core) 4 hidden layers

In [6]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_mixed_complex_symmetric_15_4h")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_mixed_complex_symmetric_15_4h


#### train

In [7]:
# number of trees in random projection forest
n_trees = 5 + int(round((train_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(train_data.shape[0]))))
# distance metric
metric = "euclidean"

In [8]:
low_train = load_encoder(train_data).cpu().numpy()

Instructions for updating:
Use tf.identity instead.


In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    t = trustworthiness(train_data, low_train, n_neighbors=int(k), metric='euclidean')
    print(k,t)

In [9]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=10
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 01:50:50 2021 Building RP forest with 16 trees
Fri Jan 29 01:50:53 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Fri Jan 29 01:51:05 2021 Building RP forest with 16 trees
Fri Jan 29 01:51:05 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.88142, 10.0, 1.0)

In [10]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 01:51:07 2021 Building RP forest with 16 trees
Fri Jan 29 01:51:09 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Fri Jan 29 01:51:11 2021 Building RP forest with 16 trees
Fri Jan 29 01:51:12 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.72616, 18.0, 1.0)

In [11]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 01:51:14 2021 Building RP forest with 16 trees
Fri Jan 29 01:51:16 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Fri Jan 29 01:51:20 2021 Building RP forest with 16 trees
Fri Jan 29 01:51:21 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(6.1169, 28.0, 1.0)

In [12]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 01:51:23 2021 Building RP forest with 16 trees
Fri Jan 29 01:51:26 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Fri Jan 29 01:51:37 2021 Building RP forest with 16 trees
Fri Jan 29 01:51:38 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(11.98314, 46.0, 1.0)

#### test

In [7]:
fitting_data =  np.concatenate((train_data, test_data), axis=0)

In [8]:
# number of trees in random projection forest
n_trees = 5 + int(round((fitting_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(fitting_data.shape[0]))))
# distance metric
metric = "euclidean"

In [9]:
low_fitting = load_encoder(fitting_data).cpu().numpy()

Instructions for updating:
Use tf.identity instead.


In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    trustworthiness = trustworthiness(fitting_data, low_fitting, n_neighbors=k, metric='euclidean')
    print(k,trustworthiness)

In [10]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 01:59:26 2021 Building RP forest with 17 trees
Fri Jan 29 01:59:30 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Fri Jan 29 01:59:43 2021 Building RP forest with 17 trees
Fri Jan 29 01:59:43 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.851, 9.0, 1.0)

In [15]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 02:01:49 2021 Building RP forest with 17 trees
Fri Jan 29 02:01:53 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Fri Jan 29 02:01:56 2021 Building RP forest with 17 trees
Fri Jan 29 02:01:56 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.61, 19.0, 1.0)

In [16]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 02:01:58 2021 Building RP forest with 17 trees
Fri Jan 29 02:02:02 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Fri Jan 29 02:02:08 2021 Building RP forest with 17 trees
Fri Jan 29 02:02:08 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(5.8616, 26.0, 1.0)

In [17]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 02:02:11 2021 Building RP forest with 17 trees
Fri Jan 29 02:02:15 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Fri Jan 29 02:02:32 2021 Building RP forest with 17 trees
Fri Jan 29 02:02:32 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(11.4128, 45.0, 1.0)

### M_<CP-CP, k=15> (core) 4 hidden layers

In [6]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_core_tptp_4h")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_core_tptp_4h


#### train

In [7]:
# number of trees in random projection forest
n_trees = 5 + int(round((train_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(train_data.shape[0]))))
# distance metric
metric = "euclidean"

In [8]:
low_train = load_encoder(train_data).cpu().numpy()

Instructions for updating:
Use tf.identity instead.


In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    t = trustworthiness(train_data, low_train, n_neighbors=int(k), metric='euclidean')
    print(k,t)

In [9]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=10
border_pres.mean(), border_pres.max(),border_pres.min()

Sat Jan 30 13:11:32 2021 Building RP forest with 16 trees
Sat Jan 30 13:11:35 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Sat Jan 30 13:11:46 2021 Building RP forest with 16 trees
Sat Jan 30 13:11:47 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.93862, 9.0, 1.0)

In [10]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Sat Jan 30 13:11:48 2021 Building RP forest with 16 trees
Sat Jan 30 13:11:51 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Sat Jan 30 13:11:53 2021 Building RP forest with 16 trees
Sat Jan 30 13:11:53 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.9074, 19.0, 1.0)

In [11]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Sat Jan 30 13:11:55 2021 Building RP forest with 16 trees
Sat Jan 30 13:11:58 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Sat Jan 30 13:12:02 2021 Building RP forest with 16 trees
Sat Jan 30 13:12:02 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(6.4148, 29.0, 1.0)

In [12]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Sat Jan 30 13:12:05 2021 Building RP forest with 16 trees
Sat Jan 30 13:12:07 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Sat Jan 30 13:12:19 2021 Building RP forest with 16 trees
Sat Jan 30 13:12:19 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(12.55232, 48.0, 1.0)

#### test

In [13]:
fitting_data =  np.concatenate((train_data, test_data), axis=0)

In [14]:
# number of trees in random projection forest
n_trees = 5 + int(round((fitting_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(fitting_data.shape[0]))))
# distance metric
metric = "euclidean"

In [15]:
low_fitting = load_encoder(fitting_data).cpu().numpy()

In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    trustworthiness = trustworthiness(fitting_data, low_fitting, n_neighbors=k, metric='euclidean')
    print(k,trustworthiness)

In [16]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Sat Jan 30 13:12:28 2021 Building RP forest with 17 trees
Sat Jan 30 13:12:31 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Sat Jan 30 13:12:33 2021 Building RP forest with 17 trees
Sat Jan 30 13:12:33 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.9104, 9.0, 1.0)

In [17]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Sat Jan 30 13:12:34 2021 Building RP forest with 17 trees
Sat Jan 30 13:12:38 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Sat Jan 30 13:12:40 2021 Building RP forest with 17 trees
Sat Jan 30 13:12:40 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.7836, 19.0, 1.0)

In [18]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Sat Jan 30 13:12:42 2021 Building RP forest with 17 trees
Sat Jan 30 13:12:46 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Sat Jan 30 13:12:50 2021 Building RP forest with 17 trees
Sat Jan 30 13:12:50 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(6.2326, 27.0, 1.0)

In [19]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Sat Jan 30 13:12:53 2021 Building RP forest with 17 trees
Sat Jan 30 13:12:56 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Sat Jan 30 13:13:10 2021 Building RP forest with 17 trees
Sat Jan 30 13:13:10 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(12.1834, 47.0, 1.0)

### M_<new_complex, k=15> (core) 4 hidden layers,symmetric

In [29]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_core_complex_symmetric_15_4h")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_core_complex_symmetric_15_4h


#### train

In [30]:
# number of trees in random projection forest
n_trees = 5 + int(round((train_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(train_data.shape[0]))))
# distance metric
metric = "euclidean"

In [31]:
low_train = load_encoder(train_data).cpu().numpy()

In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    t = trustworthiness(train_data, low_train, n_neighbors=int(k), metric='euclidean')
    print(k,t)

In [32]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=10
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 19:01:45 2021 Building RP forest with 16 trees
Fri Jan 29 19:01:48 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Fri Jan 29 19:01:49 2021 Building RP forest with 16 trees
Fri Jan 29 19:01:49 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.70944, 10.0, 1.0)

In [33]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 19:01:51 2021 Building RP forest with 16 trees
Fri Jan 29 19:01:54 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Fri Jan 29 19:01:56 2021 Building RP forest with 16 trees
Fri Jan 29 19:01:56 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.17026, 18.0, 1.0)

In [34]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 19:01:58 2021 Building RP forest with 16 trees
Fri Jan 29 19:02:01 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Fri Jan 29 19:02:06 2021 Building RP forest with 16 trees
Fri Jan 29 19:02:06 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(5.0562, 26.0, 1.0)

In [35]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 19:02:09 2021 Building RP forest with 16 trees
Fri Jan 29 19:02:11 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Fri Jan 29 19:02:24 2021 Building RP forest with 16 trees
Fri Jan 29 19:02:24 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(9.68806, 45.0, 1.0)

#### test

In [36]:
fitting_data =  np.concatenate((train_data, test_data), axis=0)

In [37]:
# number of trees in random projection forest
n_trees = 5 + int(round((fitting_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(fitting_data.shape[0]))))
# distance metric
metric = "euclidean"

In [38]:
low_fitting = load_encoder(fitting_data).cpu().numpy()

In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    trustworthiness = trustworthiness(fitting_data, low_fitting, n_neighbors=k, metric='euclidean')
    print(k,trustworthiness)

In [39]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 19:02:34 2021 Building RP forest with 17 trees
Fri Jan 29 19:02:37 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Fri Jan 29 19:02:39 2021 Building RP forest with 17 trees
Fri Jan 29 19:02:39 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.7291, 9.0, 1.0)

In [40]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 19:02:41 2021 Building RP forest with 17 trees
Fri Jan 29 19:02:44 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Fri Jan 29 19:02:47 2021 Building RP forest with 17 trees
Fri Jan 29 19:02:47 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.1632, 16.0, 1.0)

In [41]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 19:02:49 2021 Building RP forest with 17 trees
Fri Jan 29 19:02:52 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Fri Jan 29 19:02:57 2021 Building RP forest with 17 trees
Fri Jan 29 19:02:57 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(4.9981, 26.0, 1.0)

In [42]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 19:03:00 2021 Building RP forest with 17 trees
Fri Jan 29 19:03:04 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Fri Jan 29 19:03:18 2021 Building RP forest with 17 trees
Fri Jan 29 19:03:18 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(9.4624, 43.0, 1.0)

### M_<new_complex, k=30> (core) 4 hidden layers,symmetric

In [43]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_core_complex_symmetric_30_4h")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_core_complex_symmetric_30_4h


#### train

In [44]:
# number of trees in random projection forest
n_trees = 5 + int(round((train_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(train_data.shape[0]))))
# distance metric
metric = "euclidean"

In [45]:
low_train = load_encoder(train_data).cpu().numpy()

In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    t = trustworthiness(train_data, low_train, n_neighbors=int(k), metric='euclidean')
    print(k,t)

In [46]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=10
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 19:08:01 2021 Building RP forest with 16 trees
Fri Jan 29 19:08:04 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Fri Jan 29 19:08:05 2021 Building RP forest with 16 trees
Fri Jan 29 19:08:05 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.71458, 9.0, 1.0)

In [47]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 19:08:07 2021 Building RP forest with 16 trees
Fri Jan 29 19:08:10 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Fri Jan 29 19:08:12 2021 Building RP forest with 16 trees
Fri Jan 29 19:08:12 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.17656, 19.0, 1.0)

In [48]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 19:08:14 2021 Building RP forest with 16 trees
Fri Jan 29 19:08:17 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Fri Jan 29 19:08:21 2021 Building RP forest with 16 trees
Fri Jan 29 19:08:21 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(5.02842, 27.0, 1.0)

In [49]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 19:08:24 2021 Building RP forest with 16 trees
Fri Jan 29 19:08:27 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Fri Jan 29 19:08:39 2021 Building RP forest with 16 trees
Fri Jan 29 19:08:39 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(9.56968, 47.0, 1.0)

#### test

In [50]:
fitting_data =  np.concatenate((train_data, test_data), axis=0)

In [51]:
# number of trees in random projection forest
n_trees = 5 + int(round((fitting_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(fitting_data.shape[0]))))
# distance metric
metric = "euclidean"

In [52]:
low_fitting = load_encoder(fitting_data).cpu().numpy()

In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    trustworthiness = trustworthiness(fitting_data, low_fitting, n_neighbors=k, metric='euclidean')
    print(k,trustworthiness)

In [53]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 19:08:48 2021 Building RP forest with 17 trees
Fri Jan 29 19:08:52 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Fri Jan 29 19:08:53 2021 Building RP forest with 17 trees
Fri Jan 29 19:08:53 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.7423, 9.0, 1.0)

In [54]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 19:08:55 2021 Building RP forest with 17 trees
Fri Jan 29 19:08:58 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Fri Jan 29 19:09:01 2021 Building RP forest with 17 trees
Fri Jan 29 19:09:01 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.1722, 17.0, 1.0)

In [55]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 19:09:03 2021 Building RP forest with 17 trees
Fri Jan 29 19:09:06 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Fri Jan 29 19:09:11 2021 Building RP forest with 17 trees
Fri Jan 29 19:09:11 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(4.982, 27.0, 1.0)

In [56]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 19:09:13 2021 Building RP forest with 17 trees
Fri Jan 29 19:09:17 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Fri Jan 29 19:09:31 2021 Building RP forest with 17 trees
Fri Jan 29 19:09:31 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(9.4251, 46.0, 1.0)

### M_<mixed_complex, k=15> (core) 4 hidden layers,symmetric

In [57]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_mixed_complex_symmetric_15_4h")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_mixed_complex_symmetric_15_4h


#### train

In [58]:
# number of trees in random projection forest
n_trees = 5 + int(round((train_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(train_data.shape[0]))))
# distance metric
metric = "euclidean"

In [59]:
low_train = load_encoder(train_data).cpu().numpy()

In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    t = trustworthiness(train_data, low_train, n_neighbors=int(k), metric='euclidean')
    print(k,t)

In [60]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=10
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 19:11:50 2021 Building RP forest with 16 trees
Fri Jan 29 19:11:52 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Fri Jan 29 19:11:54 2021 Building RP forest with 16 trees
Fri Jan 29 19:11:54 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.90072, 9.0, 1.0)

In [61]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 19:11:56 2021 Building RP forest with 16 trees
Fri Jan 29 19:11:58 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Fri Jan 29 19:12:00 2021 Building RP forest with 16 trees
Fri Jan 29 19:12:01 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.8022, 19.0, 1.0)

In [62]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 19:12:03 2021 Building RP forest with 16 trees
Fri Jan 29 19:12:05 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Fri Jan 29 19:12:10 2021 Building RP forest with 16 trees
Fri Jan 29 19:12:10 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(6.23442, 29.0, 1.0)

In [63]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 19:12:12 2021 Building RP forest with 16 trees
Fri Jan 29 19:12:15 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Fri Jan 29 19:12:27 2021 Building RP forest with 16 trees
Fri Jan 29 19:12:27 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(12.2154, 46.0, 1.0)

#### test

In [64]:
fitting_data =  np.concatenate((train_data, test_data), axis=0)

In [65]:
# number of trees in random projection forest
n_trees = 5 + int(round((fitting_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(fitting_data.shape[0]))))
# distance metric
metric = "euclidean"

In [66]:
low_fitting = load_encoder(fitting_data).cpu().numpy()

In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    trustworthiness = trustworthiness(fitting_data, low_fitting, n_neighbors=k, metric='euclidean')
    print(k,trustworthiness)

In [67]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 19:12:36 2021 Building RP forest with 17 trees
Fri Jan 29 19:12:40 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Fri Jan 29 19:12:41 2021 Building RP forest with 17 trees
Fri Jan 29 19:12:41 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.8732, 9.0, 1.0)

In [68]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 19:12:43 2021 Building RP forest with 17 trees
Fri Jan 29 19:12:46 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Fri Jan 29 19:12:49 2021 Building RP forest with 17 trees
Fri Jan 29 19:12:49 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.6665, 19.0, 1.0)

In [69]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 19:12:51 2021 Building RP forest with 17 trees
Fri Jan 29 19:12:54 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Fri Jan 29 19:12:59 2021 Building RP forest with 17 trees
Fri Jan 29 19:12:59 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(5.9582, 27.0, 1.0)

In [70]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Fri Jan 29 19:13:02 2021 Building RP forest with 17 trees
Fri Jan 29 19:13:05 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Fri Jan 29 19:13:19 2021 Building RP forest with 17 trees
Fri Jan 29 19:13:19 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(11.5973, 45.0, 1.0)